In [0]:
from google.colab import files

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
import numpy as np
import pandas as pd

df = pd.read_csv("train.csv", index_col=0)
df.shape

(40000, 2)

In [0]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

en_stop = list(stopwords.words('english'))
lemmatizer = WordNetLemmatizer() 

def tokenize(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [t for t in tokens if
              re.match(r'[^\W\d]*$', t) and (len(t) > 2) and (t not in en_stop)]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens

tokens = df['review'].apply(tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
from gensim.models import Word2Vec

model = Word2Vec(tokens, size=300, window=10, sample=1e-5, min_count=5, iter=10, negative=20, sg=1)

In [0]:
len(model.wv.vocab)

30423

In [0]:
def encode(list_of_tokens):
    x = np.array([model.wv[t] for t in list_of_tokens if t in model.wv.vocab])
    return np.mean(x, axis=0)

fts = np.array([encode(t) for t in tokens])
fts.shape

(40000, 300)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(fts, df.label.values,
                                                    test_size=0.05, shuffle=True)

In [0]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='saga',  max_iter=3000).fit(X_train, y_train)

In [0]:
from sklearn.metrics import classification_report

predicts = clf.predict(X_train)
print('Train\n', classification_report(y_train, predicts))

predicts = clf.predict(X_test)
print('Test\n', classification_report(y_test, predicts))

Train
               precision    recall  f1-score   support

           0       0.89      0.88      0.89     19074
           1       0.88      0.89      0.89     18926

    accuracy                           0.89     38000
   macro avg       0.89      0.89      0.89     38000
weighted avg       0.89      0.89      0.89     38000

Test
               precision    recall  f1-score   support

           0       0.89      0.88      0.88       993
           1       0.88      0.89      0.88      1007

    accuracy                           0.88      2000
   macro avg       0.88      0.88      0.88      2000
weighted avg       0.88      0.88      0.88      2000



In [0]:
test = pd.read_csv('test.csv', index_col=0)


tt = test['review'].apply(tokenize)
ff = np.array([encode(t) for t in tt])


predicted = clf.predict(ff)



pd.DataFrame({'Predicted': predicted}).to_csv('solution2.csv', index_label='Id')